# Zindi Project - Financial Inclusion in Africa

Financial inclusion remains one of the main obstacles to economic and human development in Africa. For example, across Kenya, Rwanda, Tanzania, and Uganda only 9.1 million adults (or 14% of adults) have access to or use a commercial bank account.

Traditionally, access to bank accounts has been regarded as an indicator of financial inclusion. Despite the proliferation of mobile money in Africa, and the growth of innovative fintech solutions, banks still play a pivotal role in facilitating access to financial services. Access to bank accounts enable households to save and make payments while also helping businesses build up their credit-worthiness and improve their access to loans, insurance, and related services. Therefore, access to bank accounts is an essential contributor to long-term economic growth.

__The objective of this project is to create a machine learning model to predict which individuals are most likely to have or use a bank account.__ The models and solutions developed can provide an indication of the state of financial inclusion in Kenya, Rwanda, Tanzania and Uganda, while providing insights into some of the key factors driving individuals’ financial security.

# Our Goal

Our Goal is to predict how likely a person is to get a bank account. Thus 'bank_account' is our target feature.

Importing Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

Importing the data

In [2]:
# Import the data into a dataframe
test = pd.read_csv('data/Test.csv')
train = pd.read_csv('data/train.csv')

# Make a new Dataframe with all the data
df = pd.concat([test, train])

Exporting Dataframe into a csv

In [3]:
# Export the dataframe into a csv:

# Define the path to the folder in your repository
folder_path = 'data/'

# Define the file name and extension
file_name = 'data.csv'

# Concatenate the folder path and file name
file_path = f'{folder_path}/{file_name}'

# Export the DataFrame to the specified folder
df.to_csv(file_path, index=False)


## EDA Part - Exploratory Data Analysis

In this part of the notebook we look and analyze our financial inclusion data we got from Zindi.

In [4]:
# Display first 5 rows
df.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,bank_account
0,Kenya,2018,uniqueid_6056,Urban,Yes,3,30,Male,Head of Household,Married/Living together,Secondary education,Formally employed Government,NaN
1,Kenya,2018,uniqueid_6060,Urban,Yes,7,51,Male,Head of Household,Married/Living together,Vocational/Specialised training,Formally employed Private,NaN
2,Kenya,2018,uniqueid_6065,Rural,No,3,77,Female,Parent,Married/Living together,No formal education,Remittance Dependent,NaN
3,Kenya,2018,uniqueid_6072,Rural,No,6,39,Female,Head of Household,Married/Living together,Primary education,Remittance Dependent,NaN
4,Kenya,2018,uniqueid_6073,Urban,No,3,16,Male,Child,Single/Never Married,Secondary education,Remittance Dependent,NaN


In [5]:
# Print a concise summary of a DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33610 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 33610 non-null  object
 1   year                    33610 non-null  int64 
 2   uniqueid                33610 non-null  object
 3   location_type           33610 non-null  object
 4   cellphone_access        33610 non-null  object
 5   household_size          33610 non-null  int64 
 6   age_of_respondent       33610 non-null  int64 
 7   gender_of_respondent    33610 non-null  object
 8   relationship_with_head  33610 non-null  object
 9   marital_status          33610 non-null  object
 10  education_level         33610 non-null  object
 11  job_type                33610 non-null  object
 12  bank_account            23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 3.6+ MB


In [6]:
# Generate descriptive statistics
df.describe()

,year,household_size,age_of_respondent
count,33610.000000,33610.000000,33610.000000
mean,2016.975930,3.791877,38.656114
std,0.847353,2.223138,16.447127
min,2016.000000,1.000000,16.000000
25%,2016.000000,2.000000,26.000000
50%,2017.000000,3.000000,35.000000
75%,2018.000000,5.000000,49.000000
max,2018.000000,21.000000,100.000000


In [7]:
# The column labels of the DataFrame.
df.columns

Index(['country', 'year', 'uniqueid', 'location_type', 'cellphone_access',
       'household_size', 'age_of_respondent', 'gender_of_respondent',
       'relationship_with_head', 'marital_status', 'education_level',
       'job_type', 'bank_account'],
      dtype='object')

In [8]:
# Check for missing values
df.isnull().sum()

country                       0
year                          0
uniqueid                      0
location_type                 0
cellphone_access              0
household_size                0
age_of_respondent             0
gender_of_respondent          0
relationship_with_head        0
marital_status                0
education_level               0
job_type                      0
bank_account              10086
dtype: int64

In [9]:
# check for unique values in column bank_account
df['bank_account'].unique()

array([nan, 'Yes', 'No'], dtype=object)

We see that we have NaNs in the column bank account. Our goal is to fill this Nan values with values that our model (hopefully) predicts right.

What we need to do is:
* Create a data frame without NaNs. This will be the data we than will split into train and test data.
* Create a data frame with all the NaN values. This will be the data we than will have our model predict with.

In [10]:
# Create a new data frame without the NaN in our target feature
df_wo_target_nan = df.dropna()

# set new index for our dataframe without the NaNs
df_wo_target_nan = df_wo_target_nan.reset_index(drop=True)
df_wo_target_nan.isnull().value_counts()
df_wo_target_nan.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,bank_account
0,Kenya,2018,uniqueid_1,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed,Yes
1,Kenya,2018,uniqueid_2,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent,No
2,Kenya,2018,uniqueid_3,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed,Yes
3,Kenya,2018,uniqueid_4,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private,No
4,Kenya,2018,uniqueid_5,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed,No


In [11]:
# create a new data frame with only the NaN in our target feature
df_with_target_nan = df.where(df['bank_account'].isnull())
df_with_target_nan.head()

# Set a new index for our dataframe with the NaNs
df_with_target_nan = df_with_target_nan.reset_index(drop=True)
df_with_target_nan.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,bank_account
0,Kenya,2018.0,uniqueid_6056,Urban,Yes,3.0,30.0,Male,Head of Household,Married/Living together,Secondary education,Formally employed Government,NaN
1,Kenya,2018.0,uniqueid_6060,Urban,Yes,7.0,51.0,Male,Head of Household,Married/Living together,Vocational/Specialised training,Formally employed Private,NaN
2,Kenya,2018.0,uniqueid_6065,Rural,No,3.0,77.0,Female,Parent,Married/Living together,No formal education,Remittance Dependent,NaN
3,Kenya,2018.0,uniqueid_6072,Rural,No,6.0,39.0,Female,Head of Household,Married/Living together,Primary education,Remittance Dependent,NaN
4,Kenya,2018.0,uniqueid_6073,Urban,No,3.0,16.0,Male,Child,Single/Never Married,Secondary education,Remittance Dependent,NaN


At this point we now focus on our dataframe without the NaNs. 
Quick look at the dataframe:

In [12]:
df_wo_target_nan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   location_type           23524 non-null  object
 4   cellphone_access        23524 non-null  object
 5   household_size          23524 non-null  int64 
 6   age_of_respondent       23524 non-null  int64 
 7   gender_of_respondent    23524 non-null  object
 8   relationship_with_head  23524 non-null  object
 9   marital_status          23524 non-null  object
 10  education_level         23524 non-null  object
 11  job_type                23524 non-null  object
 12  bank_account            23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


In [13]:
df_wo_target_nan.describe()

,year,household_size,age_of_respondent
count,23524.000000,23524.000000,23524.000000
mean,2016.975939,3.797483,38.805220
std,0.847371,2.227613,16.520569
min,2016.000000,1.000000,16.000000
25%,2016.000000,2.000000,26.000000
50%,2017.000000,3.000000,35.000000
75%,2018.000000,5.000000,49.000000
max,2018.000000,21.000000,100.000000


In [14]:
df_wo_target_nan.head().T

,0,1,2,3,4
country,Kenya,Kenya,Kenya,Kenya,Kenya
year,2018,2018,2018,2018,2018
uniqueid,uniqueid_1,uniqueid_2,uniqueid_3,uniqueid_4,uniqueid_5
location_type,Rural,Rural,Urban,Rural,Urban
cellphone_access,Yes,No,Yes,Yes,No
household_size,3,5,5,5,8
age_of_respondent,24,70,26,34,26
gender_of_respondent,Female,Female,Male,Female,Male
relationship_with_head,Spouse,Head of Household,Other relative,Head of Household,Child
marital_status,Married/Living together,Widowed,Single/Never Married,Married/Living together,Single/Never Married


In [15]:
# Show overview of all the unique values of the dataframe:
for column in df_wo_target_nan.columns:
    unique_values = df_wo_target_nan[column].unique()
    print(f"Column '{column}' has {len(unique_values)} unique value(s):")
    print(unique_values)
    print()

Column 'country' has 4 unique value(s):
['Kenya' 'Rwanda' 'Tanzania' 'Uganda']

Column 'year' has 3 unique value(s):
[2018 2016 2017]

Column 'uniqueid' has 8735 unique value(s):
['uniqueid_1' 'uniqueid_2' 'uniqueid_3' ... 'uniqueid_8757'
 'uniqueid_8758' 'uniqueid_8759']

Column 'location_type' has 2 unique value(s):
['Rural' 'Urban']

Column 'cellphone_access' has 2 unique value(s):
['Yes' 'No']

Column 'household_size' has 20 unique value(s):
[ 3  5  8  7  1  6  4 10  2 11  9 12 16 15 13 14 21 18 17 20]

Column 'age_of_respondent' has 85 unique value(s):
[ 24  70  26  34  32  42  54  76  40  69  64  31  38  47  27  48  25  21
  18  22  58  55  62  29  35  45  67  19  80  66  50  33  28  51  16  17
  30  37  59  65  46  56  52  23  43  49  44  72  53  63  39  81  78  36
  20  60  95  71  57  85  68  41  61  75  86  73  93  74  88  90  77  84
  82  89  79  83  94  87  92  91  98  97  96  99 100]

Column 'gender_of_respondent' has 2 unique value(s):
['Female' 'Male']

Column 'relations

In [ ]:
# Show overview of all the unique values of the dataframe:
for column in df_wo_target_nan.columns:
    unique_values = df_wo_target_nan[column].unique()
    print(f"Column '{column}' has {len(unique_values)} unique value(s):")
    print(unique_values)
    print()

In [ ]:
# Checking for duplicate values
print(f"duplicate values in columns")

display(df_wo_target_nan.duplicated().value_counts())

print("______"*30)

In [ ]:
# Plotting correlation between numeric columns

import plotly.graph_objects as go

numeric_df = df_wo_target_nan.select_dtypes(include='number')
correlation_matrix = numeric_df.corr()

fig = go.Figure(data=go.Heatmap(z=correlation_matrix.values, x=correlation_matrix.columns, y=correlation_matrix.index))

fig.show()

Find out if there is a relation between the features (which contains objects) and our target feature, using the "Cramers V".

* Small Effect:
Cramér's V values close to 0 indicate a weak or negligible association between the categorical variables.

* Medium Effect:
Cramér's V values around 0.1 to 0.3 suggest a moderate association. This indicates that the variables have some degree of dependency, but the association may not be very strong.

* Large Effect:
Cramér's V values close to 0.3 or higher indicate a relatively strong association between the categorical variables. This suggests a notable dependency or relationship between the variables.
